In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


ERROR: Could not find a version that satisfies the requirement torch==1.8.1+cu111 (from versions: 2.0.0, 2.0.0+cpu, 2.0.0+cu117, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cu117, 2.0.1+cu118, 2.1.0, 2.1.0+cpu, 2.1.0+cu118, 2.1.0+cu121, 2.1.1, 2.1.1+cpu, 2.1.1+cu118, 2.1.1+cu121, 2.1.2, 2.1.2+cpu, 2.1.2+cu118, 2.1.2+cu121, 2.2.0, 2.2.0+cpu, 2.2.0+cu118, 2.2.0+cu121, 2.2.1, 2.2.1+cpu, 2.2.1+cu118, 2.2.1+cu121, 2.2.2, 2.2.2+cpu, 2.2.2+cu118, 2.2.2+cu121, 2.3.0, 2.3.0+cpu, 2.3.0+cu118, 2.3.0+cu121)
ERROR: No matching distribution found for torch==1.8.1+cu111


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

C:\Users\Mudit\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [5]:
tokens

tensor([[  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
         11838,   102]])

In [6]:
result=model(tokens)

In [7]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [14]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [17]:
int(torch.argmax(result.logits))+1

4

In [18]:
import pandas as pd
import numpy as np

In [21]:
df=pd.read_csv("C:\\Users\\Mudit\Desktop\\senti analysis\\sentiment_analysis.csv")

In [22]:
df.head()

,Year,Month,Day,Time of Tweet,text,sentiment,Platform
0,2018,8,18,morning,What a great day!!! Looks like dream.,positive,Twitter
1,2018,8,18,noon,"I feel sorry, I miss you here in the sea beach",positive,Facebook
2,2017,8,18,night,Don't angry me,negative,Facebook
3,2022,6,8,morning,We attend in the class just for listening teac...,negative,Facebook
4,2022,6,8,noon,"Those who want to go, let them go",negative,Instagram


In [24]:
df.drop(['sentiment'],axis=1)

,Year,Month,Day,Time of Tweet,text,Platform
0,2018,8,18,morning,What a great day!!! Looks like dream.,Twitter
1,2018,8,18,noon,"I feel sorry, I miss you here in the sea beach",Facebook
2,2017,8,18,night,Don't angry me,Facebook
3,2022,6,8,morning,We attend in the class just for listening teac...,Facebook
4,2022,6,8,noon,"Those who want to go, let them go",Instagram
...,...,...,...,...,...,...
494,2015,10,18,night,"According to , a quarter of families under six...",Twitter
495,2021,2,25,morning,the plan to not spend money is not going well,Instagram
496,2022,5,30,noon,uploading all my bamboozle pictures of facebook,Facebook
497,2018,8,10,night,congratulations ! you guys finish a month ear...,Twitter


In [25]:
review_store=df['text']

In [26]:
review_store

0                  What a great day!!! Looks like dream.
1         I feel sorry, I miss you here in the sea beach
2                                         Don't angry me
3      We attend in the class just for listening teac...
4                      Those who want to go, let them go
                             ...                        
494    According to , a quarter of families under six...
495        the plan to not spend money is not going well
496      uploading all my bamboozle pictures of facebook
497     congratulations ! you guys finish a month ear...
498     actually, I wish I was back in Tahoe.  I miss...
Name: text, Length: 499, dtype: object

In [29]:
review_store[0]

'What a great day!!! Looks like dream.'

In [30]:
check=tokenizer.encode(review_store[0], return_tensors='pt')

In [31]:
check

tensor([[  101, 11523,   143, 11838, 11111,   106,   106,   106, 48551, 11531,
         16121,   119,   102]])

In [32]:
check2=model(check)

In [33]:
check2

SequenceClassifierOutput(loss=None, logits=tensor([[-2.0196, -2.5199, -0.9223,  1.0585,  3.5002]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [34]:
check2.logits

tensor([[-2.0196, -2.5199, -0.9223,  1.0585,  3.5002]],
       grad_fn=<AddmmBackward0>)

In [37]:
int(torch.argmax(check2.logits))+1

5

In [38]:
new_store={}

In [39]:
def sen_score(text):
    tok=tokenizer.encode(text, return_tensors='pt')
    mod=model(tok)
    final=int(torch.argmax(mod.logits))+1
    new_store[text]=final

In [46]:
review_store

0                  What a great day!!! Looks like dream.
1         I feel sorry, I miss you here in the sea beach
2                                         Don't angry me
3      We attend in the class just for listening teac...
4                      Those who want to go, let them go
                             ...                        
494    According to , a quarter of families under six...
495        the plan to not spend money is not going well
496      uploading all my bamboozle pictures of facebook
497     congratulations ! you guys finish a month ear...
498     actually, I wish I was back in Tahoe.  I miss...
Name: text, Length: 499, dtype: object

In [47]:
for x in review_store:
    sen_score(x)

In [48]:
print("hello")

hello


In [49]:
new_store

{'What a great day!!! Looks like dream.': 5,
 'I feel sorry, I miss you here in the sea beach': 1,
 "Don't angry me": 5,
 'We attend in the class just for listening teachers reading on slide. Just Nonsence  ': 1,
 'Those who want to go, let them go': 1,
 'Its night 2 am, feeling neutral': 2,
 '2 am feedings for the baby are fun when he is all smiles and coos': 4,
 'Soooo high': 3,
 ' Both of you': 5,
 'Today first time I arrive in the boat. Its amazing journey': 5,
 'Love is something like E=MC^2 rules': 5,
 'I really really like the song Cardigan by Taylor Swift': 4,
 'My Sharpie is running DANGERously low on ink': 1,
 'i want to go to music tonight but i lost my voice.': 2,
 'Always somewhere I miss you where I have been': 5,
 "Don't distrube me": 5,
 "I don't care, who are you and what can you do": 5,
 'i`ve been sick for the past few days ': 1,
 "I'm come back home, my past job, its really good time for me": 5,
 'I will never fall in love until I found you': 5,
 ' oh Maya, I`m so s

In [51]:
new_dataframe=pd.DataFrame(new_store,index=[0])

In [55]:
new_dataframe.T

,0
What a great day!!! Looks like dream.,5
"I feel sorry, I miss you here in the sea beach",1
Don't angry me,5
We attend in the class just for listening teachers reading on slide. Just Nonsence,1
"Those who want to go, let them go",1
...,...
"According to , a quarter of families under six live in poverty.",3
the plan to not spend money is not going well,1
uploading all my bamboozle pictures of facebook,5
congratulations ! you guys finish a month early than we do. booo,5
